In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler as MMS

In [2]:
from itertools import product

In [30]:
parameter_space = {'n_estimators':[2,1000],
                  'max_depth':[2,40],
                  'min_samples_split':[2,100],
                  'min_samples_leaf':[2,200],
                  }
  

In [27]:
m = MMS().fit(df)

In [28]:
m.transform(df)

array([[0.61954181, 0.71352877, 0.93780482],
       [0.71439395, 0.        , 0.        ],
       [1.        , 1.        , 0.15881686],
       [0.        , 0.26693869, 1.        ]])

In [29]:
df

,X_0,X_1,X_2
0,0.728462,0.424559,0.766475
1,0.791278,0.130102,0.209185
2,0.980420,0.542779,0.303562
3,0.318171,0.240262,0.803435


In [3]:
dimensions = 3
N = 4

In [4]:
df = pd.DataFrame(data=np.random.uniform(size=(N,dimensions)), columns=[f'X_{i}' for i in range(dimensions)])
df;

In [31]:
len(parameter_space)

4

In [33]:
mms = MMS().fit(pd.DataFrame(parameter_space))
mms

MinMaxScaler()

In [34]:
pd.DataFrame(columns=parameter_space.keys())

,n_estimators,max_depth,min_samples_split,min_samples_leaf


In [98]:
class Minesweeper:
    def __init__(self, parameter_space):
        self.b = 3
        self.df = pd.DataFrame(columns=parameter_space.keys())
        self.mms = MMS().fit(pd.DataFrame(parameter_space))
        self.dimensions = len(parameter_space)
        
    @property
    def bins(self):
        return np.linspace(0,1,self.b+1)
    
    def get_parameters(self):
        while True:
            all_cells = set(product(list(range(self.b)), repeat=self.dimensions))
            occupied_cells = list(set(self.df.apply(lambda x:np.digitize(x, self.bins)-1, 
                                           result_type='reduce', axis =1).apply(tuple)))
        
            relative_neighbors = (list(product((-1,0,1), repeat=self.dimensions)))
            exclude_cells = list(set([(tuple([i+j for i,j in zip(a,b)])) for b in relative_neighbors for a in occupied_cells]))
            exclude_cells = set([x for x in exclude_cells if all(0<=i<self.b for i in x)])
        
            vacant_cells = list(all_cells-exclude_cells)
            if len(vacant_cells)==0:
                self.b+=1
                continue
                
            chosen_cell = vacant_cells[np.random.randint(0,len(vacant_cells))]
            print(self.b, chosen_cell)
            chosen_values = tuple([(self.bins[i]+self.bins[i+1])/2 for i in chosen_cell])
            return self.mms.inverse_transform([chosen_values])[0]
        
    def update_values(self,values):
        values = pd.DataFrame({col:[value] for col, value in zip(self.mms.feature_names_in_, values)})
        new_values = pd.DataFrame(columns=self.df.columns, data=self.mms.transform(values))
        self.df = pd.concat([self.df, new_values], ignore_index=True)

In [99]:
ms= Minesweeper(parameter_space)
ms

In [100]:
ms.get_parameters()

3 (1, 1, 2, 1)


array([501.        ,  21.        ,  83.66666667, 101.        ])

In [101]:
ms.df

,n_estimators,max_depth,min_samples_split,min_samples_leaf


In [111]:
new_values = ms.get_parameters()
print(new_values)
ms.update_values(new_values)
ms.df

4 (0, 0, 3, 2)
[126.75   6.75  87.75 125.75]


,n_estimators,max_depth,min_samples_split,min_samples_leaf
0,0.500000,0.500000,0.500000,0.833333
1,0.166667,0.500000,0.833333,0.166667
2,0.833333,0.500000,0.500000,0.166667
3,0.166667,0.166667,0.166667,0.166667
4,0.166667,0.833333,0.166667,0.166667
5,0.625000,0.875000,0.125000,0.375000
6,0.625000,0.125000,0.125000,0.875000
7,0.375000,0.875000,0.625000,0.125000
8,0.375000,0.625000,0.125000,0.875000
9,0.125000,0.125000,0.875000,0.625000


In [71]:
ms.mms.inverse_transform(ms.df)

array([[833.66666667,  21.        ,  83.66666667, 167.        ],
       [501.        ,  21.        ,  18.33333333, 167.        ],
       [168.33333333,  33.66666667,  83.66666667, 167.        ],
       [168.33333333,   8.33333333,  18.33333333,  35.        ],
       [833.66666667,   8.33333333,  51.        ,  35.        ],
       [168.33333333,   8.33333333,  83.66666667, 167.        ],
       [833.66666667,  33.66666667,  18.33333333,  35.        ]])

In [ ]:
ms.df

In [ ]:
b = 3
bins = np.linspace(0,1,b+1)
bins;

In [ ]:
# df.apply(lambda x:np.digitize(x,bins)-1, axis=1, result_type='reduce').apply(tuple)

In [ ]:
all_cells = set(product( list(range(b)), repeat=dimensions))
all_cells;

In [ ]:
occupied_cells = list(set(df.apply(lambda x:np.digitize(x, bins)-1, result_type='reduce', axis =1).apply(tuple)))
occupied_cells;

In [ ]:
relative_neighbors = (list(product((-1,0,1), repeat=dimensions)))
relative_neighbors[:10];

In [ ]:
exclude_cells = list(set([(tuple([i+j for i,j in zip(a,b)])) for b in relative_neighbors for a in occupied_cells]))
exclude_cells = set([x for x in exclude_cells if all(0<=i<b for i in x)]);

In [ ]:
exclude_cells;

In [ ]:
len(exclude_cells)

In [ ]:
len(all_cells)

In [ ]:
len(all_cells - exclude_cells)

In [ ]:
all_cells-exclude_cells

In [ ]:
vacant_cells = list(all_cells-exclude_cells)
chosen_cell = vacant_cells[np.random.randint(0,len(vacant_cells))]

In [ ]:
chosen_cell

In [ ]:
chosen_values = tuple([(bins[i]+bins[i+1])/2 for i in chosen_cell])

In [ ]:
chosen_values